In [1]:
import pandas as pd
import numpy as np
import os
from datetime import time,timedelta
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from scipy.stats import spearmanr, pearsonr


In [2]:
head_dir = './data'
# days示例：['1','2']
# stock示例：'A'
def generate_and_process_data(days:list[str],stock:str,head_dir:str = head_dir):
    
    data_daylist = []
    days = sorted(days)

    for day in days:
        df = pd.read_csv(f'{head_dir}/{day}/{stock}.csv') #读取

        df['LastPrice'] = df['LastPrice'].replace(0,np.nan).ffill() #将零值用前一个非零值填充
        df = df.rename(columns={'BidVolum5':'BidVolume5',
                                'AskVolum5':'AskVolume5'}) #修改了几个出错的列名，看着难受
        data_daylist.append(df)
        
    return data_daylist
    



In [3]:
A_list = generate_and_process_data(['1'],'A')
A_df = A_list[0]

print(A_df['Time'].dtype)

def exist_exception(mask:pd.Series):
    mark = True
    for i in mask:
        if not i:
            print('存在异常值')
            mark = False
            break
    if mark:
        print('不存在异常值')

exist_exception(A_df['AskPrice1'] > A_df['BidPrice1'])
exist_exception((A_df['LastPrice'] >= A_df['BidPrice1']) & (A_df['LastPrice'] <= A_df['AskPrice1'])) #由于之前的前向填充处理，这里确实会出现异常值
exist_exception(A_df['LastPrice'] != 0)


int64
不存在异常值
存在异常值
不存在异常值


In [4]:
all_days = ['1','2','3','4','5']

# list是以天为单位的列表，每个元素是一个dataframe
A_list = generate_and_process_data(all_days,'A')
B_list = generate_and_process_data(all_days,'B')
C_list = generate_and_process_data(all_days,'C')
D_list = generate_and_process_data(all_days,'D')
E_list = generate_and_process_data(all_days,'E')

# 由于一天的刚开始订单量异常多，故分开考虑开盘时5分钟前后的数据
A_list_5min = [df[df['Time'] < 93500000] for df in A_list]
B_list_5min = [df[df['Time'] < 93500000] for df in B_list]
C_list_5min = [df[df['Time'] < 93500000] for df in C_list]
D_list_5min = [df[df['Time'] < 93500000] for df in D_list]
E_list_5min = [df[df['Time'] < 93500000] for df in E_list]

A_list_rest = [df[df['Time'] >= 93500000] for df in A_list]
B_list_rest = [df[df['Time'] >= 93500000] for df in B_list]
C_list_rest = [df[df['Time'] >= 93500000] for df in C_list]
D_list_rest = [df[df['Time'] >= 93500000] for df in D_list]
E_list_rest = [df[df['Time'] >= 93500000] for df in E_list]

# 将5天的数据拼到一起，形成一个dataframe
A_df_5min_all = pd.concat(A_list_5min)
B_df_5min_all = pd.concat(B_list_5min)
C_df_5min_all = pd.concat(C_list_5min)
D_df_5min_all = pd.concat(D_list_5min)
E_df_5min_all = pd.concat(E_list_5min)

A_df_rest_all = pd.concat(A_list_rest)
B_df_rest_all = pd.concat(B_list_rest)
C_df_rest_all = pd.concat(C_list_rest)
D_df_rest_all = pd.concat(D_list_rest)
E_df_rest_all = pd.concat(E_list_rest)


滚动标准化可能带来的问题是，因子会受极端值影响，从而可能整体失真，AI建议我们用mad法，去除极端值，可以一试

这是一个非常深刻且切中要害的问题。您完全正确，滚动标准化虽然必要，但确实会带来“受极端值影响而导致因子值失真”的风险。这正是在实盘因子工程中必须严肃对待和妥善处理的核心问题之一。
下面我们来详细拆解这个问题，并提供解决方案。
为什么滚动标准化会受极端值影响？
想象一个简单的滚动Z-Score标准化公式：
因子值_标准化 = (因子值_t - 滚动窗口内因子均值) / 滚动窗口内因子标准差
在某个时刻t，一个巨大的异常值（极端值）进入了滚动窗口。
影响1：拉偏均值：这个异常值会显著拉高或拉低窗口内的均值。
影响2：炸大标准差：由于标准差衡量的是波动，这个异常值会使得窗口内的标准差急剧增大。
双重打击：在接下来的一个窗口期内，所有正常因子值的标准化计算都会基于这个被扭曲的均值和被放大的标准差。导致：
原本正常的因子值会被压缩到一个非常小的区间内（例如，全部被压缩到[-0.5, 0.5]），失去了区分度。
标准化后的序列会出现一个明显的“凹陷”或“平台”，这完全是由单个数据错误引起的，而非市场的真实变化。
这种情况在高频数据中尤其致命，比如一笔“乌龙指”交易或者一个数据错误，就可能导致几分钟内的因子失效。

解决方案：从“均值-标准差”到“稳健标准化”
解决方案的核心是：在滚动标准化之前或之中，使用对异常值不敏感的“稳健统计量”来替代均值和标准差。
以下是几种行之有效的方法，按推荐顺序排列：

方法一：在标准化前先处理极端值（去极值）
这是最常用、最有效的流程。先对原始因子序列进行去极值处理，然后再进行滚动标准化。
MAD法（中位数绝对偏差法）
思想：用中位数代替均值，用绝对偏差代替标准差。这两个统计量对异常值都不敏感。
步骤：
计算滚动窗口内因子值的中位数 MEDIAN。
计算每个因子值与中位数的绝对偏差：|因子值 - MEDIAN|。
计算这些绝对偏差的中位数，得到 MAD。
找到一个稳健的“标准差异常值”判断阈值，通常为：MEDIAN ± k * MAD（常见的k取3或5）。
将超过阈值的因子值截断（Winsorization）或设为中位数。
优势：非常稳健，即使窗口内存在多个异常值也能较好处理。
分位数法
思想：直接使用因子分布的分位数来判定异常值。
步骤：将因子值限制在滚动窗口的某个分位数区间内，例如 [1%分位数, 99%分位数] 或 [5%分位数, 95%分位数]。超出部分用分位数值代替。
代码示例（MAD去极值 + 滚动标准化）：
def robust_rolling_zscore(series, window, k=5):
    """
    稳健的滚动标准化：先MAD去极值，再Z-Score
    """
    # 1. 计算滚动中位数和MAD
    rolling_median = series.rolling(window).median()
    rolling_mad = (series - rolling_median).abs().rolling(window).median()
    
    # 2. 计算上下限，并进行去极值（Winsorization）
    upper_bound = rolling_median + k * rolling_mad
    lower_bound = rolling_median - k * rolling_mad
    # 将超出界限的值用界限值替代
    winsorized_series = series.clip(lower=lower_bound, upper=upper_bound)
    
    # 3. 对去极值后的序列进行标准的滚动Z-Score标准化
    # 使用去极值后序列的滚动均值和标准差
    rolling_mean_win = winsorized_series.rolling(window).mean()
    rolling_std_win = winsorized_series.rolling(window).std()
    
    return (winsorized_series - rolling_mean_win) / rolling_std_win
    
方法二：直接使用稳健的标准化方法
不先处理原始值，而是在标准化公式中直接使用稳健统计量。
基于中位数和MAD的标准化
公式：因子值_标准化 = (因子值_t - 滚动中位数) / (滚动MAD)
注意：为了使“MAD”与“标准差”在尺度上可比，通常会将MAD乘以一个常数（约1.4826），因为正态分布中 标准差 ≈ 1.4826 * MAD。
修正公式：因子值_标准化 = (因子值_t - 滚动中位数) / (滚动MAD * 1.4826)
方法三：使用次序统计量进行标准化
思想：不依赖具体的数值分布，而是基于排名。
方法：在滚动窗口内，将因子值转换为其在窗口内的分位数排名或正态化分位数。例如，使用scipy.stats.rankdata或pandas.DataFrame.rank计算出每个点的分位数（介于0和1之间），然后再映射到标准正态分布的分位数上。这种方法完全不受异常值影响，但可能会损失一些数值间的线性关系。
实战建议与工作流
可视化！可视化！可视化！
将原始因子序列、滚动均值/标准差序列、以及标准化后的因子序列同时画出来。
异常值导致的失真在图上会非常明显（比如标准差突然出现一个尖峰，随后因子被压缩）。
建立标准化流水线：
首选：方法一，即“去极值 -> 标准化”的流水线。这是业界的黄金标准。
参数选择：对于去极值的参数（如MAD的k值），需要结合因子的历史分布进行回测确定。通常k=3对应较严格的标准，k=5更宽松。
效果评估：
一个稳健的标准化方法，其输出的因子序列应该具有相对稳定的波动性（例如，滚动标准差不会出现剧烈的、短暂的跳跃）。
最终，还是要看经过稳健处理后的因子在样本外测试中的IC序列是否更稳定，IR（信息比率）是否得到提升。
总结
方法
	
优点
	
缺点
	
适用场景


传统Z-Score​
	
计算简单，意义直观
	
对异常值极度敏感
	
数据非常干净，或仅用于探索性分析


MAD去极值+Z-Score​
	
非常稳健，效果好，业界主流​
	
计算稍复杂，需选择参数k
	
绝大多数场景，强烈推荐​


中位数+MAD标准化​
	
无需去极值步骤，一步到位
	
对极端密集分布可能稍敏感
	
对实时性要求极高，可接受轻微失真


分位数标准化​
	
完全不受异常值影响，结果稳定
	
损失数值间的部分信息，计算量大
	
因子值与收益率为单调但非线性的关系
结论：您的担忧是完全必要的。​ 直接使用传统的滚动Z-Score标准化是危险的。请务必在标准化前加入稳健的去极值处理（如MAD法），这样可以极大程度地免疫于极端值的影响，保证因子在生产环境中的稳定性和可靠性。

In [5]:
#滚动标准化：
def rolling_zscore(series:pd.Series,
                   window_z:int,
):
    
    series_mean = series.rolling(window_z,min_periods=1).mean().shift(1)
    series_std = series.rolling(window_z,min_periods=1).std().shift(1)+1e-8
    series_std = series_std.replace(0,np.nan).fillna(method="bfill")
    series_zscore = (series-series_mean)/series_std
    return series_zscore

def rolling_minmax(series:pd.Series,
                   window_z:int
):

    series_min = series.rolling(window_z,min_periods=1).min().shift(1)
    series_max = series.rolling(window_z,min_periods=1).max().shift(1)
    series_minmax = (series-series_min)/(series_max-series_min+1e-8)
    return series_minmax

# 稳健的滚动标准化：
def mad_rolling_zscore(series:pd.Series,
                       window:int,
                       k=5):
    """
    稳健的滚动标准化：先MAD去极值，再Z-Score
    """
    # 1. 计算滚动中位数和MAD
    rolling_median = series.rolling(window,min_periods=1).median()
    rolling_mad = (series - rolling_median).abs().rolling(window,min_periods=1).median()
    
    # 2. 计算上下限，并进行去极值（Winsorization）
    upper_bound = rolling_median + k * rolling_mad
    lower_bound = rolling_median - k * rolling_mad
    # 将超出界限的值用界限值替代
    winsorized_series = series.clip(lower=lower_bound, upper=upper_bound)
    
    # 3. 对去极值后的序列进行标准的滚动Z-Score标准化
    # 使用去极值后序列的滚动均值和标准差
    rolling_mean_win = winsorized_series.rolling(window,min_periods=1).mean()
    rolling_std_win = winsorized_series.rolling(window,min_periods=1).std()
    
    return (winsorized_series - rolling_mean_win) / rolling_std_win


In [8]:
# 回溯区间内的平均价格和回溯区间起始价格之间计算一个收益率
def cal_PastReturn(lastprice:pd.Series,
                   window:int):
    base = lastprice.shift(window).fillna(method='bfill')
    ave = lastprice.rolling(window,min_periods=1).mean()
    ret = ave/base - 1
    return ret

# 回溯区间内成交价的高低价差和当前成交价的比值
def cal_HLR(lastprice:pd.Series,
                   window:int):
    high = lastprice.rolling(window,min_periods=1).max()
    low = lastprice.rolling(window,min_periods=1).min()
    hlr = (high - low)/lastprice

    return hlr

# 回溯区间买卖价差的标准差
def cal_BSV(bidprice:pd.Series,
            askprice:pd.Series,
            window:int):
    minus = bidprice-askprice
    bsv = minus.rolling(window,min_periods=1).std()
    return bsv

# 下行收益率的波动率：只考虑相对前一刻下跌的tick的跌幅,计算其标准差
def cal_DownLogreturn_std(lastprice:pd.Series,
                      window:int):
    logreturn = np.log(lastprice).diff()
    downlogreturn = logreturn.where(logreturn < 0,np.nan).rolling(window,min_periods=1).std()

    return downlogreturn

# 下行收益率的均值
def cal_DownLogreturn(lastprice:pd.Series,
                      window:int):
    logreturn = np.log(lastprice).diff()
    downlogreturn = logreturn.where(logreturn < 0,np.nan).rolling(window,min_periods=1).mean()

    return downlogreturn

# 上行收益率的波动率：只考虑相对前一刻上涨的tick的跌幅,计算其标准差
def cal_UpLogreturn_std(lastprice:pd.Series,
                      window:int):
    logreturn = np.log(lastprice).diff()
    uplogreturn = logreturn.where(logreturn > 0,np.nan).rolling(window,min_periods=1).std()

    return uplogreturn

# 上行收益率的均值
def cal_UpLogreturn(lastprice:pd.Series,
                    window:int):
    logreturn = np.log(lastprice).diff()
    uplogreturn = logreturn.where(logreturn > 0,np.nan).rolling(window,min_periods=1).mean()

    return uplogreturn

# 对成交价进行时间加权平均（ewm指数加权，越近的时间权重越大)后，长时间窗口和短时间窗口的差
def cal_DIF(lastprice:pd.Series,
            window_long:int,
            window_short:int):
    long = lastprice.ewm(span=window_long,adjust=False).mean()
    short = lastprice.ewm(span=window_short,adjust=False).mean()
    dif = long - short

    return dif

# 对dif进行平均
def cal_DEA(lastprice:pd.Series,
            window_long:int,
            window_short:int,
            window_dea:int):
    
    dea = cal_DIF(lastprice,window_long,window_short).rolling(window_dea,min_periods=1).mean()

    return dea

# 滚动成交量加权均价（不用lastprice而是amount和volume的比值）
def cal_VWAP(sellamount:pd.Series,
             buyamount:pd.Series,
             sellvolume:pd.Series,
             buyvolume:pd.Series,
             window:int):
    sumamount = sellamount + buyamount
    sumvolume = sellvolume + buyvolume
    vwap = sumamount.rolling(window,min_periods=1).sum() / sumvolume.rolling(window,min_periods=1).sum()

    return vwap

# 差分后用ewm算涨跌比值，不用RSI
def cal_RS(lastprice:pd.Series,
            window:int):
    gain = lastprice.where(lastprice > 0,0).ewm(span=window,adjust=False).mean()
    loss = -lastprice.where(lastprice < 0,0).ewm(span=window,adjust=False).mean()
    rs = gain / (loss + 1e-8)

    return rs

# AI推荐的RSI的差分，或许可以衡量动能
def cal_RSImomentum(lastprice:pd.Series,
                    window:int):
    rs = cal_RS(lastprice,window)
    rsi = 100 - 100/(1 + rs)
    rsi_momentum = rsi.diff().bfill().rolling(window,min_periods=1).mean()

    return rsi_momentum

#计算VOI和OIR所用的五档权重函数
def weigh(
        df:pd.DataFrame,
        buy_list:list=["BidVolume1","BidVolume2","BidVolume3","BidVolume4","BidVolume5"],
        sell_list:list=["AskVolume1","AskVolume2","AskVolume3","AskVolume4","AskVolume5"],
        switch_buyorsell:str="s" ,
        decay:int = 0.5   
        ):
    data = df.copy()
    data_sum = pd.Series(0,index=data.index)
    if switch_buyorsell == "s":
        for i,index in enumerate(sell_list):
            data_1 = data[index] * np.exp(-decay*i)
            data_sum += data_1
    elif switch_buyorsell == "b":
        for i,index in enumerate(buy_list):
            data_1 = data[index] * np.exp(-decay*i)
            data_sum += data_1
    return data_sum/ np.sum([np.exp(-decay * i) for i in range(5)]) #归一化

''' 计算订单失衡情况，加权后的五档买量和五档卖量的差
def cal_OBI(df:pd.DataFrame,
            window:int):
    buyorder = weigh(df,switch_buyorsell="b").rolling(window,min_periods=1).mean()
    sellorder = weigh(df,switch_buyorsell="s").rolling(window,min_periods=1).mean()
    obi = buyorder - sellorder
    
    return obi
但是经过IC测试，发现这个指标的表现很差，可能是计算方式不对，或者这个指标本身就不太有用，暂时先放弃了
'''
# 换一种计算订单失衡情况的方式，不用五档量，而用挂单买量和卖量的差
def cal_OBI(orderbuy:pd.Series,
            ordersell:pd.Series,
            window:int):
    obi = (orderbuy - ordersell).rolling(window,min_periods=1).mean()

    return obi

# 订单失衡比（同样换一种方式）
def cal_OIR(orderbuy:pd.Series,
            ordersell:pd.Series,
            window:int):
    buyorder = orderbuy.rolling(window,min_periods=1).mean()
    sellorder = ordersell.rolling(window,min_periods=1).mean()
    oir = (buyorder - sellorder) / (buyorder + sellorder + 1e-8)

    return oir

# 主动买入和卖出量之差
def cal_VOI(orderbuyvolume:pd.Series,
            ordersellvolume:pd.Series,
            window:int):
    voi = (orderbuyvolume - ordersellvolume).rolling(window,min_periods=1).mean()

    return voi

In [18]:
# 计算平均订单量失衡情况：平均买单量和平均卖单量的差
def cal_AVO(orderbuyvolume:pd.Series,
            ordersellvolume:pd.Series,
            orderbuynum:pd.Series,
            ordersellnum:pd.Series,
            window:int):
    
    avbuy = orderbuyvolume / (orderbuynum + 1e-8)
    avsell = ordersellvolume / (ordersellnum + 1e-8)
    avo = (avbuy - avsell).rolling(window,min_periods=1).mean()
    return avo

# 订单失衡和买卖价差的结合指标：订单失衡乘以买卖价差的标准差
def cal_BSVmultiOBI(orderbuy:pd.Series,
            ordersell:pd.Series,
            bidprice:pd.Series,
            askprice:pd.Series,
            window:int):
    obi = cal_OBI(orderbuy,ordersell,window)
    bsv = cal_BSV(bidprice,askprice,window)
    bsvmultiobi = bsv * obi
    return bsvmultiobi

# 上行收益率和下行收益率的结合指标：二者的差值
def cal_UpDownLogreturn(lastprice:pd.Series,
                        window:int):
    downlogreturn = cal_DownLogreturn(lastprice,window)
    uplogreturn = cal_UpLogreturn(lastprice,window)
    updownlogreturn = uplogreturn + downlogreturn
    return updownlogreturn

In [9]:
def calculate_ic_series(factor_series: pd.Series, 
                       return_series: pd.Series, 
                       method: str = 'spearman', # 默认斯皮尔曼相关系数
                       min_periods: int = 30) -> pd.Series:
    """
    计算因子值与未来收益率之间的IC序列
    
    参数:
    ----------
    factor_series : pd.Series
        因子值序列，索引应为时间戳
    return_series : pd.Series
        未来收益率序列，索引应与factor_series一致
    method : str, 可选 ('spearman', 'pearson')
        相关系数计算方法:
        - 'spearman': 斯皮尔曼秩相关系数（推荐，对异常值不敏感）
        - 'pearson': 皮尔逊相关系数（要求数据近似正态分布）
    min_periods : int
        计算相关系数所需的最小样本数
    
    返回:
    -------
    pd.Series
        IC值序列，索引与输入序列一致
    """
    
    # 检查输入数据
    if not isinstance(factor_series, pd.Series) or not isinstance(return_series, pd.Series):
        raise TypeError("输入必须是pandas Series")
    
    if len(factor_series) != len(return_series):
        raise ValueError("因子序列和收益率序列长度必须一致")
    
    # 对齐索引
    aligned_data = pd.DataFrame({
        'factor': factor_series,
        'return': return_series
    }).dropna()
    
    if len(aligned_data) == 0:
        raise ValueError("对齐后的数据为空，请检查输入序列")
    
    # 根据方法选择计算函数
    if method == 'spearman':
        def corr_func(x, y):
            return spearmanr(x, y).correlation
    elif method == 'pearson':
        def corr_func(x, y):
            return pearsonr(x, y)[0]
    else:
        raise ValueError("method参数必须是'spearman'或'pearson'")
    
    # 计算IC序列
    ic_values = []
    dates = []
    
    # 按时间顺序计算滚动IC
    for i in range(min_periods, len(aligned_data)):
        current_data = aligned_data.iloc[:i+1]
        
        # 确保有足够的数据点
        if len(current_data) >= min_periods:
            try:
                ic = corr_func(current_data['factor'].values, 
                              current_data['return'].values)
                ic_values.append(ic)
                dates.append(aligned_data.index[i])
            except (ValueError, TypeError):
                # 处理计算错误（如所有值相同的情况）
                ic_values.append(np.nan)
                dates.append(aligned_data.index[i])
    
    return pd.Series(ic_values, index=dates, name=f'IC_{method}')

def analyze_ic_series(ic_series: pd.Series) -> dict:
    """
    对IC序列进行统计分析
    
    参数:
    ----------
    ic_series : pd.Series
        calculate_ic_series函数返回的IC序列
    
    返回:
    -------
    dict
        包含IC统计指标的字典
    """
    ic_clean = ic_series.dropna()
    
    if len(ic_clean) == 0:
        return {"error": "无有效IC值"}
    
    stats = {
        'IC_mean': ic_clean.mean(),           # IC均值
        'IC_std': ic_clean.std(),             # IC标准差
        'IC_IR': ic_clean.mean() / ic_clean.std() if ic_clean.std() != 0 else 0,  # 信息比率
        'IC_positive_ratio': (ic_clean > 0).mean(),  # IC为正的比例
        'IC_significant_ratio': (ic_clean.abs() > 0.02).mean(),  # |IC| > 0.02的比例
        'IC_abs_mean': ic_clean.abs().mean(),  # IC绝对值均值
        'observation_count': len(ic_clean),   # 有效观测数
        'total_periods': len(ic_series)       # 总期数
    }
    
    return stats

In [10]:
E_lastprice_rest_all = E_df_rest_all['LastPrice']
target_rest = E_df_rest_all['Return5min']

In [ ]:

bsv_factor =  cal_BSV(E_df_rest_all['BidPrice1'],E_df_rest_all['AskPrice1'],window = 600)
bsv_ic = calculate_ic_series(bsv_factor,target_rest)

print(analyze_ic_series(bsv_ic))

{'IC_mean': np.float64(-0.09093616075893289), 'IC_std': 0.11202381942780285, 'IC_IR': np.float64(-0.8117573675261043), 'IC_positive_ratio': np.float64(0.00584557025797561), 'IC_significant_ratio': np.float64(0.8611732629987998), 'IC_abs_mean': np.float64(0.09193411682354057), 'observation_count': 134974, 'total_periods': 134974}


In [13]:
pastreturn_factor = cal_PastReturn(E_lastprice_rest_all,window = 600)


In [14]:
pastreturn_ic = calculate_ic_series(pastreturn_factor,target_rest)
print(analyze_ic_series(pastreturn_ic))

{'IC_mean': np.float64(-0.21023994791669584), 'IC_std': 0.1493329191287198, 'IC_IR': np.float64(-1.4078606990564237), 'IC_positive_ratio': np.float64(0.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.21023994791669584), 'observation_count': 134975, 'total_periods': 134975}


In [12]:
hlr_factor = cal_HLR(E_lastprice_rest_all,window = 600)
hlr_ic = calculate_ic_series(hlr_factor,target_rest)
print(analyze_ic_series(hlr_ic))

{'IC_mean': np.float64(-0.11490123419981563), 'IC_std': 0.15259118796723842, 'IC_IR': np.float64(-0.75300045651709), 'IC_positive_ratio': np.float64(0.022885719577699574), 'IC_significant_ratio': np.float64(0.8764956473421004), 'IC_abs_mean': np.float64(0.13656868140377515), 'observation_count': 134975, 'total_periods': 134975}


In [10]:
random_factor = pd.Series(np.random.randn(len(target_rest)),index=target_rest.index)
random_ic = calculate_ic_series(random_factor,target_rest)
print(analyze_ic_series(random_ic))

{'IC_mean': np.float64(-0.0025588853244871406), 'IC_std': 0.004899966990000531, 'IC_IR': np.float64(-0.5222250128845998), 'IC_positive_ratio': np.float64(0.17772920911279866), 'IC_significant_ratio': np.float64(0.0032672717169846266), 'IC_abs_mean': np.float64(0.003202259123265408), 'observation_count': 134975, 'total_periods': 134975}


In [11]:
downlogreturnstd_factor = cal_DownLogreturn_std(E_lastprice_rest_all,window = 600)
downlogreturnstd_ic = calculate_ic_series(downlogreturnstd_factor,target_rest)
print(analyze_ic_series(downlogreturnstd_ic))

{'IC_mean': np.float64(-0.11609899886389814), 'IC_std': 0.1369944806134042, 'IC_IR': np.float64(-0.8474720904379155), 'IC_positive_ratio': np.float64(0.0012225029451207313), 'IC_significant_ratio': np.float64(0.916573435381458), 'IC_abs_mean': np.float64(0.11772556095970274), 'observation_count': 134969, 'total_periods': 134969}


In [12]:
downlogreturn_factor = cal_DownLogreturn(E_lastprice_rest_all,window = 600)
downlogreturn_ic = calculate_ic_series(downlogreturn_factor,target_rest)
print(analyze_ic_series(downlogreturn_ic))

{'IC_mean': np.float64(0.10580664377771039), 'IC_std': 0.1461737084943664, 'IC_IR': np.float64(0.7238418240020792), 'IC_positive_ratio': np.float64(0.8885900570497147), 'IC_significant_ratio': np.float64(0.6436689634733644), 'IC_abs_mean': np.float64(0.10834022329107298), 'observation_count': 134970, 'total_periods': 134970}


In [13]:
uplogreturnstd_factor = cal_UpLogreturn_std(E_lastprice_rest_all,window = 600)
uplogreturnstd_ic = calculate_ic_series(uplogreturnstd_factor,target_rest)
print(analyze_ic_series(uplogreturnstd_ic))

{'IC_mean': np.float64(-0.09051951001998458), 'IC_std': 0.13915654274927852, 'IC_IR': np.float64(-0.6504869137419979), 'IC_positive_ratio': np.float64(0.16823246991879556), 'IC_significant_ratio': np.float64(0.6128934265900066), 'IC_abs_mean': np.float64(0.09597836947002249), 'observation_count': 134968, 'total_periods': 134968}


In [14]:
uplogreturn_factor = cal_UpLogreturn(E_lastprice_rest_all,window = 600)
uplogreturn_ic = calculate_ic_series(uplogreturn_factor,target_rest)
print(analyze_ic_series(uplogreturn_ic))

{'IC_mean': np.float64(-0.09362565187949709), 'IC_std': 0.14783473447972964, 'IC_IR': np.float64(-0.6333129504983457), 'IC_positive_ratio': np.float64(0.2366639006608778), 'IC_significant_ratio': np.float64(0.5999985182111846), 'IC_abs_mean': np.float64(0.10151181469380549), 'observation_count': 134972, 'total_periods': 134972}


In [10]:
dif_factor = cal_DIF(E_lastprice_rest_all,window_long = 600,window_short=300)
dif_ic = calculate_ic_series(dif_factor,target_rest)
print(analyze_ic_series(dif_ic))

{'IC_mean': np.float64(0.25404924558949166), 'IC_std': 0.13319721261484882, 'IC_IR': np.float64(1.9073165316461755), 'IC_positive_ratio': np.float64(1.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.25404924558949166), 'observation_count': 134975, 'total_periods': 134975}


In [11]:
dea_factor = cal_DEA(E_lastprice_rest_all,window_long = 600,window_short=300,window_dea=150)
dea_ic = calculate_ic_series(dea_factor,target_rest)
print(analyze_ic_series(dea_ic))

{'IC_mean': np.float64(0.24844639653147377), 'IC_std': 0.1347328100061809, 'IC_IR': np.float64(1.8439932821120277), 'IC_positive_ratio': np.float64(1.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.24844639653147377), 'observation_count': 134975, 'total_periods': 134975}


In [13]:
vwap_factor = cal_VWAP(E_df_rest_all['TradeSellAmount'],E_df_rest_all['TradeBuyAmount'],
                       E_df_rest_all['TradeSellVolume'],E_df_rest_all['TradeBuyVolume'],window=600)
vwap_ic = calculate_ic_series(vwap_factor,target_rest)
print(analyze_ic_series(vwap_ic))

{'IC_mean': np.float64(-0.16531167626854926), 'IC_std': 0.13663244496093355, 'IC_IR': np.float64(-1.2099005936387641), 'IC_positive_ratio': np.float64(0.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.16531167626854926), 'observation_count': 134975, 'total_periods': 134975}


In [14]:
rs_factor = cal_RS(E_lastprice_rest_all,window=600)
rs_ic = calculate_ic_series(rs_factor,target_rest)
print(analyze_ic_series(rs_ic))

{'IC_mean': np.float64(-0.18227422023556875), 'IC_std': 0.1524276785207788, 'IC_IR': np.float64(-1.1958078874154165), 'IC_positive_ratio': np.float64(0.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.18227422023556875), 'observation_count': 134975, 'total_periods': 134975}


In [17]:
rsimomentum_factor = cal_RSImomentum(E_lastprice_rest_all,window=600)
rsimomentum_ic = calculate_ic_series(rsimomentum_factor,target_rest)
print(analyze_ic_series(rsimomentum_ic))

{'IC_mean': np.float64(-0.2345901123578258), 'IC_std': 0.1441830769833014, 'IC_IR': np.float64(-1.6270294494061526), 'IC_positive_ratio': np.float64(0.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.2345901123578258), 'observation_count': 134975, 'total_periods': 134975}


In [9]:
obi_factor = cal_OBI(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
obi_ic = calculate_ic_series(obi_factor,target_rest)
print(analyze_ic_series(obi_ic))

{'IC_mean': np.float64(-0.15996495077893572), 'IC_std': 0.148719490380236, 'IC_IR': np.float64(-1.0756152429647794), 'IC_positive_ratio': np.float64(0.0004000740877940359), 'IC_significant_ratio': np.float64(0.9995184293387664), 'IC_abs_mean': np.float64(0.16007054730776862), 'observation_count': 134975, 'total_periods': 134975}


In [11]:
oir_factor = cal_OIR(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
oir_ic = calculate_ic_series(oir_factor,target_rest)
print(analyze_ic_series(oir_ic))

{'IC_mean': np.float64(-0.1425737449361117), 'IC_std': 0.15913563408869916, 'IC_IR': np.float64(-0.895925954941329), 'IC_positive_ratio': np.float64(0.0012224486015928876), 'IC_significant_ratio': np.float64(0.9998147805149101), 'IC_abs_mean': np.float64(0.14280645110141013), 'observation_count': 134975, 'total_periods': 134975}


In [23]:
voi_factor = cal_VOI(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
voi_ic = calculate_ic_series(voi_factor,target_rest)
print(analyze_ic_series(voi_ic))

{'IC_mean': np.float64(-0.15996495077893572), 'IC_std': 0.148719490380236, 'IC_IR': np.float64(-1.0756152429647794), 'IC_positive_ratio': np.float64(0.0004000740877940359), 'IC_significant_ratio': np.float64(0.9995184293387664), 'IC_abs_mean': np.float64(0.16007054730776862), 'observation_count': 134975, 'total_periods': 134975}


In [13]:
avo_factor = cal_AVO(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],E_df_rest_all['OrderBuyNum'],E_df_rest_all['OrderSellNum'],window=600)
avo_ic = calculate_ic_series(avo_factor,target_rest)
print(analyze_ic_series(avo_ic))

{'IC_mean': np.float64(-0.13279016098832486), 'IC_std': 0.11750694008589367, 'IC_IR': np.float64(-1.130062283055449), 'IC_positive_ratio': np.float64(3.704389701796629e-05), 'IC_significant_ratio': np.float64(0.9999851824411928), 'IC_abs_mean': np.float64(0.13279463401141117), 'observation_count': 134975, 'total_periods': 134975}


In [15]:
bsvmultiobi_factor = cal_BSVmultiOBI(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],E_df_rest_all['BidPrice1'],E_df_rest_all['AskPrice1'],window=600)
bsvmultiobi_ic = calculate_ic_series(bsvmultiobi_factor,target_rest)
print(analyze_ic_series(bsvmultiobi_ic))

{'IC_mean': np.float64(-0.16006856641187164), 'IC_std': 0.14417031302614142, 'IC_IR': np.float64(-1.1102741129711462), 'IC_positive_ratio': np.float64(0.0002815357031724628), 'IC_significant_ratio': np.float64(0.9999925911657059), 'IC_abs_mean': np.float64(0.16030698401788906), 'observation_count': 134974, 'total_periods': 134974}


In [19]:
updownlogreturn_factor = cal_UpDownLogreturn(E_lastprice_rest_all,window=600)
updownlogreturn_ic = calculate_ic_series(updownlogreturn_factor,target_rest)
print(analyze_ic_series(updownlogreturn_ic))

{'IC_mean': np.float64(0.12913081795369794), 'IC_std': 0.1041451852372689, 'IC_IR': np.float64(1.2399115490505441), 'IC_positive_ratio': np.float64(1.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.12913081795369794), 'observation_count': 134970, 'total_periods': 134970}
